In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在 GCP 上的端到端 ML：MLOps 阶段 5：使用 Vertex AI 私有端点开始
<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage5/get_started_with_vertex_private_endpoints.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage5/get_started_with_vertex_private_endpoints.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage5/get_started_with_vertex_private_endpoints.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>          
</table>
<br/><br/><br/>

## 概述

此教程演示了如何使用Vertex AI SDK创建和使用`Vertex AI Endpoint`资源来为模型提供服务。`Vertex AI Private Endpoints`是一种私有端点，为客户端和服务器之间提供点对点网络gRPC通信（即内部网络），在同一网络中。这消除了使用HTTP协议（即互联网）的公共端点的网络切换和路由开销。

了解更多关于[私有端点](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints)。

### 目标

在本教程中，您将学习如何使用`Vertex AI Private Endpoint`资源。

此教程使用以下谷歌云ML服务和资源：

- `Vertex AI Endpoints`
- `Vertex AI Models`
- `Vertex AI Prediction`

执行的步骤包括：

- 创建`Private Endpoint`资源。
- 配置VPC互连连接。
- 配置`Model`资源的serving二进制文件以部署到`Private Endpoint`资源。
- 将`Model`资源部署到`Private Endpoint`资源。
- 向`Private Endpoint`发送预测请求。

#### `Private Endpoint`设置与`Public Endpoint`有何不同

- 启用两个额外的API：Service Networking和Cloud DNS。
- 将Compute Admin Network角色添加到您（默认）的服务帐户。
- 发出两个gcloud命令来为您的服务帐户设置VPC互连。
- *目前*还没有SDK支持，因此私有终端点是使用GAPIC客户端创建的，并且有一个额外的参数用于互连网络。
- 要发送请求，您不能使用SDK/GAPIC，因为它们会执行HTTP互联网请求。相反，您使用curl发送对等请求。

*未来计划是支持终结点和使用点对点协议进行预测请求的SDK支持。*

数据集

本教程使用了来自 TensorFlow Hub 的预训练图像分类模型，该模型是在 ImageNet 数据集上训练的。

了解更多关于[ResNet V2 预训练模型](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5)。

### 成本

本教程使用了 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 基于您的预期使用量生成成本估算。

## 安装

安装以下包以执行这个笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG -q
! pip3 install tensorflow-hub $USER_FLAG -q
! pip3 install --upgrade tensorflow $USER_FLAG -q

重新启动内核

安装额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU 运行环境

*如果您有此选项，请确保在 GPU 运行环境中运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的谷歌云项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个账户时，您将获得 $300 的免费信用额，用于您的计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、Compute Engine API 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. [启用服务网络 API](https://console.cloud.google.com/flows/enableapi?apiid=servicenetworking.googleapis.com)。

5. [启用 Cloud DNS API](https://console.cloud.google.com/flows/enableapi?apiid=dns.googleapis.com)。

6. 如果您是在本地运行此笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

7. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 在此笔记本中的所有命令中使用正确的项目。

**注意**: Jupyter 会将以 `!` 为前缀的行作为 shell 命令运行，并会对以 `$` 为前缀的 Python 变量进行内插。

#### 设置您的项目 ID

**如果您不知道您的项目 ID**，您可以使用 `gcloud` 获取您的项目 ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于笔记本的其余部分的操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶进行使用Vertex AI进行训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您参加现场教程会话，您可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账户

**如果您正在使用Vertex AI Workbench笔记本**，则您的环境已经经过验证。跳过此步骤。

**如果您使用的是Colab**，运行下面的单元格，并按提示进行身份验证以通过oAuth进行账户验证。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务账户对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，并选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，并选择**存储桶对象管理员**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中，将您的服务账户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个云存储桶

**无论您的笔记本环境如何，都需要执行以下步骤。**

当您为 Python 初始化 Vertex AI SDK 时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源在不同会话间保留的地方。

在下方设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查Cloud Storage存储桶中的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip
import tensorflow as tf
import tensorflow_hub as hub

### 初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化用于Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`DEPLOY_GPU / DEPLOY_NGPU`设置为使用支持GPU的容器映像，以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个 GPU 容器映像，并将每个 VM 分配4个 Nvidia Telsa K80 GPU，您可以指定：

(aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，请指定`(None, None)`以使用一个在 CPU 上运行的容器映像。

详细了解[您地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注*：TF 2.3 之前的 GPU 支持版本不能在本教程中加载自定义模型将失败。这是一个已知问题，在 TF 2.3 中已修复。这是由于在服务功能中生成的静态图操作。如果您在自己的自定义模型上遇到此问题，请使用具有 GPU 支持的 TF 2.3 容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于预测的预构建Docker容器镜像。 

要获取最新列表，请参阅[预构建的预测容器]（https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers）。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置将用于预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存。
     - `n1-highcpu`：每个vCPU 0.9GB内存。
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：您还可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

## 从TensorFlow Hub获取预训练模型

为了演示目的，本教程使用来自TensorFlow Hub（TFHub）的预训练模型，然后将其上传到`Vertex AI Model`资源。一旦你拥有了`Vertex AI Model`资源，该模型就可以部署到`Vertex AI Private Endpoint`资源。

### 下载预训练模型

首先，你需要从TensorFlow Hub下载预训练模型。该模型将以TF.Keras层的形式下载。在这个示例中，为了完成模型，你可以创建一个带有下载的TFHub模型作为层的`Sequential()`模型，并指定模型的输入形状。

In [ ]:
tfhub_model = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model.build([None, 224, 224, 3])

tfhub_model.summary()

### 保存模型工件

此刻，模型已经在内存中。接下来，您将模型工件保存到云存储位置。

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"
tfhub_model.save(MODEL_DIR)

上传用于服务的模型

接下来，您将从自定义作业将TF.Keras模型上传到Vertex `Model`服务，该服务将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个服务函数来将数据转换为模型期望的格式。如果您发送编码数据到Vertex AI，则您的服务函数确保数据在传递到模型输入之前在模型服务器上解码。

### 服务函数如何工作

当您向在线预测服务器发送请求时，请求将由HTTP服务器接收。HTTP服务器从HTTP请求内容正文中提取预测请求。提取的预测请求将被转发到服务函数。对于Google预构建的预测容器，请求内容将作为`tf.string`传递给服务函数。

服务函数包括两个部分：

- `预处理函数`：
  - 将输入(`tf.string`)转换为底层模型(dynamic graph)期望的输入形状和数据类型。
  - 执行与训练底层模型期间相同的数据预处理--例如，归一化，缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序期望的格式--例如，压缩输出。
  - 为接收应用程序打包输出--例如，添加标题，制作JSON对象等。

预处理和后处理函数都被转换为与模型融合的静态图。从底层模型的输出传递到后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在为TF.Keras模型构建服务函数时需考虑的一个因素是它们作为静态图运行。这意味着您不能使用需要动态图的TF图操作。如果这样做，您将在编译服务函数时收到一个错误，指示您正在使用不受支持的EagerTensor。

### 图像数据的服务功能

#### 预处理

要将图像传递给预测服务，您需要将压缩（例如JPEG）图像字节编码为base 64 - 这样可以使内容在通过网络传输二进制数据时不会被修改。由于此部署模型期望输入数据为原始（未压缩）字节，您需要确保将base 64编码的数据转换回原始字节，然后对其进行预处理以匹配模型输入要求，然后将其作为输入传递给部署的模型。

为解决此问题，您需要定义一个服务功能（`serving_fn`）并将其附加到模型作为预处理步骤。添加`@tf.function`装饰器，以便服务函数融合到基础模型（而不是在CPU上游）。

当您发送预测或解释请求时，请求的内容将被base 64解码为一个Tensorflow字符串（`tf.string`），然后传递给服务函数（`serving_fn`）。服务函数将`tf.string`预处理为原始（未压缩）的numpy字节（`preprocess_fn`）以符合模型的输入要求：

- `io.decode_jpeg`- 解压缩JPG图像，返回一个具有三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值更改为float 32，并将像素数据重新缩放为0到1之间。
- `image.resize` - 将图像大小调整为与模型输入形状匹配。

此时，数据可以通过具体函数传递到模型（`m_call`）。服务功能是一个静态图，而模型是一个动态图。具体函数执行从服务函数到模型的输入数据的编配任务，并将模型的预测结果从模型返回到服务功能。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(224, 224))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(tfhub_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(tfhub_model, MODEL_DIR, signatures={"serving_default": serving_fn})

获取服务函数签名

您可以通过重新加载模型到内存中，并查询与每个层对应的签名来获取模型的输入和输出层的签名。

为了您的目的，您需要服务函数的签名。为什么呢？当我们将数据发送给模型进行预测时作为HTTP请求数据包时，图像数据被base64编码，而我们的TF.Keras模型需要numpy输入。您的服务函数将会从base64转换为一个numpy数组。

在进行预测请求时，您需要将请求路由到服务函数而不是模型，因此您需要知道服务函数的输入层名称--这将在您之后进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(MODEL_DIR)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 将TensorFlow Hub模型上传到“Vertex AI模型”资源

最后，您将TFHub模型中的模型工件上传到“Vertex AI模型”资源中。

In [ ]:
model = aip.Model.upload(
    display_name="example_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(model)

## 设置VPC对等网络

为了使用“私有端点”，您需要在您的项目和托管运行您的模型的“Vertex AI预测”服务项目之间建立VPC对等网络。这消除了网络流量中的额外跳跃，并允许使用高效的gRPC协议。

了解更多关于[VPC对等](https://cloud.google.com/vertex-ai/docs/general/vpc-peering)。

**重要提示：您每个项目只能建立一个到servicenetworking.googleapis.com的VPC对等连接。**

### 为默认网络创建VPC对等连接

为了简单起见，我们设置了VPC对等连接到默认网络。您可以为您的项目创建一个不同的网络。

如果您将VPC对等连接设置到任何其他网络，请确保该网络已经存在并且您的虚拟机正在运行在该网络上。

In [ ]:
# This is for display only; you can name the range anything.
PEERING_RANGE_NAME = "vertex-ai-prediction-peering-range"
NETWORK = "default"

# NOTE: `prefix-length=16` means a CIDR block with mask /16 will be
# reserved for use by Google services, such as Vertex AI.
! gcloud compute addresses create $PEERING_RANGE_NAME \
  --global \
  --prefix-length=16 \
  --description="peering range for Google service" \
  --network=$NETWORK \
  --purpose=VPC_PEERING

### 创建VPC连接

接下来，为VPC对等连接创建连接。

*注意:* 如果出现权限拒绝错误，可能是因为您的默认服务账户没有设置必要的角色“Compute Network Admin”。在云控制台中，按照以下步骤操作。

1. 进入 `IAM & Admin`
2. 找到您的服务账户。
3. 点击编辑图标。
4. 选择 `添加其他角色`。
5. 输入“Compute Network Admin”。
6. 点击 `保存`。

In [ ]:
! gcloud services vpc-peerings connect \
  --service=servicenetworking.googleapis.com \
  --network=$NETWORK \
  --ranges=$PEERING_RANGE_NAME \
  --project=$PROJECT_ID

检查您的对等连接状态。

In [ ]:
! gcloud compute networks peerings list --network $NETWORK

构建完整的网络名称

在随后为VPC peering创建`Private Endpoint`资源时，您需要拥有完整的网络资源名称。

In [ ]:
project_number = model.resource_name.split("/")[1]
print(project_number)

full_network_name = f"projects/{project_number}/global/networks/{NETWORK}"

## 创建`Endpoint`资源

您可以使用`PrivateEndpoint.create()`方法创建一个`Endpoint`资源。

在本例中，指定了以下参数：

- `display_name`：`Private Endpoint`资源的可读名称。
- `network`：VPC peering的完整网络资源名称。

了解更多关于[Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aip.PrivateEndpoint.create(
    display_name="private_" + TIMESTAMP, network=full_network_name
)

获取有关“Endpoint”资源的详细信息

您可以使用属性“gca_resource”获取“Endpoint”对象的底层详细信息。

In [ ]:
endpoint.gca_resource

## 部署 `Model` 资源到一个 `Endpoint` 资源。

您可以将一个或多个 `Vertex AI Model` 资源实例部署到同一个端点上。每个部署的 `Vertex AI Model` 资源都会有自己的用于服务二进制文件的部署容器。

*注意:* 在前面上传模型文件到 `Vertex AI Model` 资源的步骤中，您已经为 TFHub 模型指定了部署容器。

### 部署单个 `Endpoint` 资源

在下一个例子中，您将部署一个 `Vertex AI Model` 资源到一个 `Vertex AI Endpoint` 资源。 在部署时，您需要指定以下附加配置设置:

- 机器类型。
- GPU 的类型和数量（如果有）。
- VM 实例的静态、手动或自动扩展。

对于 `Private Endpoint`，您只能部署一个模型。因此，没有流量分割。

在此示例中，您将使用指定参数的最小化模型部署，如下所示:

- `model`: `Model` 资源。
- `deployed_model_displayed_name`: 部署模型实例的人类可读名称。
- `machine_type`: 每个 VM 实例的机器类型。

由于资源需求，此部署可能需要几分钟。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint)

获取已部署模型的信息

您可以从`Endpoint`资源配置数据`gca_resource.deployed_models`中获取已部署模型的部署设置。在此示例中，只部署了一个模型，因此有关下标`[0]`的提到。

In [ ]:
print(endpoint.gca_resource.deployed_models[0])

### 为预测准备测试数据

接下来，您将加载一个压缩的JPEG图像到内存中，然后对其进行base64编码。为了演示目的，您将使用来自Flowers数据集的一张图像。

In [ ]:
! gsutil cp gs://cloud-ml-data/img/flower_photos/daisy/100080576_f52e8ee070_n.jpg test.jpg

In [ ]:
import base64

with open("test.jpg", "rb") as f:
    data = f.read()
b64str = base64.b64encode(data).decode("utf-8")

### 进行预测

现在您的 `Model` 资源已部署到 `Endpoint` 资源中，您可以通过向 Endpoint 资源发送预测请求来进行在线预测。

#### 请求

由于 `Private Endpoint` 会阻止来自公共HTTP请求的请求，您将使用 `curl` 发送请求到私有URI。

首先，您将构建请求为JSON文件。为了将测试数据传递给预测服务，您将字节编码为base64 - 这使得内容在通过网络传输二进制数据时免受修改。

每个实例的格式为：

    { serving_input: { 'b64': base64编码的字节 } }

由于serving二进制可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目列表发送。

In [ ]:
import json

with open("instances.json", "w") as f:
    f.write(json.dumps({"instances": [{serving_input: {"b64": b64str}}]}))

### 使用 SDK 发出预测请求

接下来，使用 `Vertex AI SDK` 发出预测请求。

In [ ]:
instances = [{serving_input: {"b64": b64str}}]

prediction = endpoint.predict(instances=instances)
print(prediction)

### 从`Endpoint`资源中撤销`Model`资源

当一个`Model`资源被部署到一个`Endpoint`资源时，部署的`Model`资源实例会被分配一个ID，通常被称为部署的模型ID。

您可以使用`undeploy()`方法撤销特定的`Model`资源实例，需要以下参数：

- `deployed_model_id`：分配给部署模型的ID。

In [ ]:
deployed_model_id = endpoint.gca_resource.deployed_models[0].id
print(deployed_model_id)

endpoint.undeploy(deployed_model_id)

最后，您可以使用`undeploy_all()`方法从`Endpoint`资源中取消部署所有`Model`实例。

In [ ]:
endpoint.undeploy_all()

### 删除一个 `Endpoint` 资源

如果 `Endpoint` 资源没有部署模型，则可以使用 `delete()` 方法删除 `Endpoint` 资源。

In [ ]:
endpoint.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

In [ ]:
delete_bucket = False
delete_model = True

if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}